In [1]:
# load Python modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

from plot_helpers import generate_pdf_panel
from plot_helpers import plot_pdf
from plot_helpers import plot_pdf_and_cdf

In [ ]:
# # stats
# mean, var, skew, kurt = rvF.stats(moments='mvsk')
# mean, var, skew, kurt


### Why do we need probability theory in statistics?

Probability theory provides us with some essential tools needed for statistics:
  - modelling: describe a population as a probability distribution
  - sampling distributions: distributions obtained for random samples from a population

Additionally, artificially introduced randomness is the basis for two of the most important statistics "kung fu" techniques:

- **random selection** = if you collect a sample of size n randomly selected from a population, then compute statistics from the sample, you can then make inferences about the whole population.
- **random assignment** = create two roughly equivalent groups by randomly assigning participants into group A or group B. This random split is the basis for a statistical experiment that compares an intervention group and a control group. Given the participants were randomly assigned to the two groups, we can hope they are roughly equally distributed on all dimensions (similar distribution of all confounding variables), so that any differences observed between groups can be attributed to the intervention. Calculating the difference statistic intervention-control give an estimate of the effect size of this intervention.

# CUT MATERIAL from 21_prob_theory.ipynb

In [ ]:
# load Python modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Figures setup
sns.set_theme(
    context="paper",
    style="whitegrid",
    palette="colorblind",
    rc={'figure.figsize': (7,4)},
)

%config InlineBackend.figure_format = 'retina'

# # silence annoying warnings
# import warnings; warnings.filterwarnings('ignore')

In [ ]:
# set random seed for repeatability
np.random.seed(42)

### Computing prob integrals using quad

#### Uniform

In [ ]:
def fU(u):
    if 0 <= u and u <= 1:
        return 1.0
    else:
        return 0.0

In [ ]:
from scipy.integrate import quad
quad(fU, -10, 10)[0]

In [ ]:
quad(fU, 0.2, 0.5)[0]

#### Normal

In [ ]:
mu = 1000
sigma = 100

def fN(n):
    z = (n - mu)/sigma
    C = sigma * np.sqrt(2*np.pi)
    return 1 / C * np.exp(-1/2 * z**2)

In [ ]:
quad(fN, 0, 2000)[0]

We're using limits of integration 0 and 2000 because that function is practically zero outside this range, and function `quad` [needs](https://stackoverflow.com/questions/63673369/while-using-scipy-integrate-quad-increasing-bound-of-a-positive-function-decrea) finite limits of integration to work properly.

In [ ]:
quad(fN, 800, 900)[0]

### Computing prob integrals using trapz

In [ ]:
from scipy.integrate import trapz

In [ ]:
m = 1000
ns = np.linspace(800, 900, m)
fNs = fN(ns)

trapz(fNs, ns)

In [ ]:
# how different 
quad(fN, 800, 900)[0] - trapz(fNs, ns)

## Using prob models

In [ ]:
# uniform distribution U(0,1)
from sympy import symbols, integrate, Integral, simplify
x, a, b = symbols('x a b')
fU = 1

simplify( integrate(fU, (x,a,b)) )

In [ ]:
Integral(fU, x)

In [ ]:
Integral(fU, x).doit()

## Random process examples using Python (bonus material)

Let's look at some simple examples of two random events: a coin flip and a die roll.
We'll use the Python module `random` to generate random numbers.

In [ ]:
import random

random.random() # random number between 0 (inclusive) and 1 (non-inclusive)

### Coin flip example

The code example below shows a function that performs the equivalent of a coin flip.

In [ ]:
def flip_coin():
    r = random.random()  # a number in the range [0,1)
    if r < 0.5:
        return "heads"
    else:
        return "tails"

flip_coin()

#### Flipping a coin many times

Let's see what happens if we flip the coin many times.

In [ ]:
flips = [flip_coin() for _ in range(0, 2000)]
heads = [flip for flip in flips if flip == "heads"]

len(heads)/len(flips)

# plot a horizontal bar chart of the number of outcomes
s = pd.Series(flips)
print(s.value_counts().sort_index())
s.value_counts().sort_index().plot(kind="barh")

### Six-sided die example

In [ ]:
def roll_die():
    r = random.random()*6  # a number in the range [0,6)
    rint = int(r)          # convert to integer in {0,1,2,3,4,5}
    value = rint + 1       # convert to integer in {1,2,3,4,5,6}
    return value

roll_die()

Let's see what happens if we roll the die many times.

In [ ]:

rolls = [roll_die() for _ in range(0, 6000)]
ds = pd.Series(rolls)
print(ds.value_counts().sort_index())
ds.value_counts().sort_index().plot(kind="bar")

#### Intentionally skipped

In [ ]:
rvN.moment(1)  # = rvN.mean()

In [ ]:
# Let's verify the formula:
# E[(x-𝜇)^2]  = E[X^2]         -   E[X]^2
#               second moment      first moment squared
rvN.var()    == rvN.moment(2)  -   rvN.moment(1)**2

In [ ]:
from scipy.stats import norm

# 1. choose the parameters:
mean = 1000   # 𝜇 (mu)    = where is its center?
std = 100     # 𝜎 (sigma) = how spread out is it?

# 2. create a normal random variable
rvN = norm(mean, std)

# 3. plot it's probability density function (pdf)
x = np.linspace(rvN.ppf(0.001), rvN.ppf(0.999), 10000)
y = rvN.pdf(x)
ax = sns.lineplot(x=x, y=y)
title = 'Probability density for the normal distribution $\mathcal{N}('+ str(mean) + ',' + str(std) + ')$'
ax.set_title(title)
ax.set_xlim(300,1700)
plt.xticks(np.arange(400,1700,100))
_ = ax.set_ylim(0, 0.006)

In [ ]:
rvN.entropy()

In [ ]:
0.5*np.log(2*np.pi*sigma**2) + 0.5

#### EARLY ATTEMPTS AT EXPECTATIONS CALCULATIONS

In [ ]:
from sympy import Rational
integrate( (u-Rational(1,2))**2, (u,0,1) )

In [ ]:
from scipy.stats import uniform

xs = np.linspace(0, 1, 1000)
rvU = uniform(0, 1)

In [ ]:
from scipy.integrate import trapz

# mean
y1 = xs*rvU.pdf(xs)
mu = trapz(y1, xs)
mu

In [ ]:
# variance
y2 = (xs-mu)**2 * rvU.pdf(xs)
sigma2 = trapz(y2, xs)
sigma2

In [ ]:
rvU.var()

#### Alt

In [ ]:
from scipy.stats import uniform

rvU = uniform(0, 1)

def xpU(x):
    return x*rvU.pdf(x)

quad(xpU, 0, 1)[0]

In [ ]:
def xpU(x):
    muX = rvU.mean()
    return x*rvU.pdf(x)

quad(xpU, 0, 1)[0]

In [ ]:
from scipy.integrate import simpson

xs = np.linspace(0, 1, 1000)
rvU = uniform(0, 1)


simpson(xs*rvU.pdf(xs), xs)

### Example probability calculation

What is the probability of observing a value between 800 and 900.

In [ ]:
from plot_helpers import calc_prob_and_plot

from scipy.stats import norm

rvN = norm(1000, 100)

In [ ]:
from scipy.integrate import quad

# Pr(800 < X < 900) = integral of f_X between 800 and 900
quad(rvN.pdf, 800, 900)[0]

In [ ]:
calc_prob_and_plot(rvN, 600, 1000)

In [ ]:
# what type of object is rvH?
type(rvH)

The above line tells us some information about the internal implementation of the Python object `rvH`,
which is the Python implementation of the random variable $H \sim \textrm{Pois}(20)$.
The `rvH` object is an instance of the class `rv_frozen` (frozen random variable).
Ignore the frozen part. The key part is that it's a random variable.